In [8]:
import collections
import json
import datetime
import pandas as pd

with open("/Users/nnayak/orda_annotations_0516.json", 'r') as f:
    orda_data = json.load(f)
    
with open ("/Users/nnayak/orda_text_0415.json", 'r') as f:
    orda_text = json.load(f)

# with open("/Users/nnayak/orda_assignments_0505.json", 'r') as f:
#     orda_assignments = json.load(f)

In [9]:
review_annotation_map = collections.defaultdict(list)

for review_annotation in orda_data["reviewannotation"]:
    review_annotation_map[review_annotation["fields"]["review_id"]].append(review_annotation["fields"]["initials"])
    

rebuttal_annotation_map = collections.defaultdict(set)

for rebuttal_annotation in orda_data["reviewannotation"]:
    rebuttal_annotation_map[rebuttal_annotation["fields"]["review_id"]].add(rebuttal_annotation["fields"]["initials"])
    

In [10]:
assert set(review_annotation_map.keys()) == set(rebuttal_annotation_map.keys())

In [11]:
review_id_to_dataset = {}
review_id_to_forum_id = {}
forum_to_reviews = collections.defaultdict(list)
for example in orda_text["example"]:
    review_id_to_dataset[example["fields"]["review_id"]] = example["fields"]["dataset"]
    review_id_to_forum_id[example["fields"]["review_id"]] = example["fields"]["forum_id"]
    forum_to_reviews[example["fields"]["forum_id"]].append(example["fields"]["review_id"])

records = []
for review_id in review_annotation_map.keys():
    records.append(
    {
        "review_id": review_id,
        "forum_id": review_id_to_forum_id[review_id],
        "dataset": review_id_to_dataset[review_id],
        "overall_dataset": review_id_to_dataset[review_id].split("_")[0],
        "review_annotators": "|".join(sorted(review_annotation_map[review_id])),
        "rebuttal_annotators": "|".join(sorted(rebuttal_annotation_map[review_id])),
        "review_annotations": len(review_annotation_map[review_id]),
        "rebuttal_annotations": len(rebuttal_annotation_map[review_id]),
    })
    
r = pd.DataFrame.from_dict(records)

In [12]:
r.to_csv("0516_annotations.csv")

In [13]:
r

,review_id,forum_id,dataset,overall_dataset,review_annotators,rebuttal_annotators,review_annotations,rebuttal_annotations
0,rJgLrhZq2X,BylctiCctX,traindev_train,traindev,AS|MC|PKY|TJO|TJO,AS|MC|PKY|TJO,5,4
1,example_review,example_forum,example,example,AN|AS|CB|KG|MAD|MC|MS|NNK|RG|SM|TJO,AN|AS|CB|KG|MAD|MC|MS|NNK|RG|SM|TJO,11,11
2,HJeXDu9h2X,S1g9N2A5FX,traindev_train,traindev,MC|TJO,MC|TJO,2,2
3,B1lChPhJpQ,BylctiCctX,traindev_train,traindev,MC|PKY|TJO|TJO,MC|PKY|TJO,4,3
4,H1gVYFVS3X,HylTBhA5tQ,traindev_train,traindev,MC|PKY,MC|PKY,2,2
...,...,...,...,...,...,...,...,...
450,B1eVKwraFH,SkgTR3VFvH,truetest,truetest,KG,KG,1,1
451,Skx8qjXJcr,B1xybgSKwB,truetest,truetest,KG,KG,1,1
452,rkgfiCmT5r,rklhqkHFDB,truetest,truetest,KG,KG,1,1
453,SJx0zByAYr,H1gEP6NFwr,truetest,truetest,MAD,MAD,1,1


In [14]:
annotated_review_set = set(review_annotation_map.keys())

for forum, reviews in forum_to_reviews.items():
    annotated_reviews = set(reviews).intersection(annotated_review_set)
    if not annotated_reviews:
        continue
    if annotated_reviews == set(reviews):
        continue
    to_annotate = set(reviews) - annotated_review_set
    for review_id in to_annotate:
        print(forum, review_id)
#         for annotator in "AS CB MC PKY".split():
#             print(review_id, annotator)

ByxPYjC5KQ S1lCpY4dn7
ryxSrhC9KX SJxwLXOY27
rkxQ-nA9FX S1gZOpsFnQ
ByME42AqK7 rJeAPV55hQ
ryeyti0qKX ByxWllVI2Q
Bklzkh0qFm rkx6mDnd37
Bklzkh0qFm BkemLDvAn7
Bkeuz20cYm BkxiXMw927
HkMlGnC9KQ SylRu6annQ
Sklv5iRqYX SkgS2HI5hQ
HJfQrs0qt7 HJl5RsCF3m
B1fpDsAqt7 SJgyyPTv3m
B1fpDsAqt7 H1gUmqkh3Q
ryepUj0qtX rJldfOy-27
SyxYEoA5FX Hkl2yixyCX
SyxYEoA5FX rkg7S_D93X
Syx72jC9tm SyeASuDp27
ryxjH3R5KQ rylJA7G82Q
ryxLG2RcYX B1ehFbba37
HJeQbnA5tm BkgYfE5o3Q
HJeQbnA5tm SJllCbbahQ
HyNmRiCqtm rJx5ghGShX
SJz6MnC5YQ S1l4P2HuhX
B14ejsA5YQ BJlOnG0gTX
B14ejsA5YQ BkeWgLUpnX
ByGOuo0cYm S1xGt0Rq3m
SJMO2iCct7 rJgBQzFjnX
SJMO2iCct7 Bygvx6sc2X
rkxtl3C5YX SJlYT1Uw27
SyMras0cFQ ryxPs1nNnQ
r1lrAiA5Ym ryxWDI_Gsm
rkeSiiA5Fm HygPsSJLpQ
SJNceh0cFX SkeyBm0VnQ
HkfYOoCcYX SJgAEEpDhQ
HkfYOoCcYX HJlAUngO2X
H1ERcs09KQ HJx9wwdJaQ
rygk9oA9Ym H1lDjH9p3X
rygk9oA9Ym BJgJHwjq2Q
rJlJ-2CqtX BkxWG9hlTQ
SkeQniAqK7 BylH3HnQnm
SkeQniAqK7 SklwBiVQaQ
BJGVX3CqYm H1gK_Y3J6Q
BJGVX3CqYm H1guYU4w67
Skf-oo0qt7 BkxRG1XIhX
ryxhynC9KX rJxd44Xt27
ryxhynC9KX